[View in Colaboratory](https://colab.research.google.com/github/taisuke0812/tweet_analysis_word2vec/blob/master/Tweet_word2vec.ipynb)



*   必要なライブラリをインストールする




In [0]:
!pip install janome
!pip install gensim

ツイートをリストに変換する

In [0]:
import pandas as pd
import re
import numpy as np
import glob
from janome.tokenizer import Tokenizer
from gensim.models import word2vec

def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(",".join(text))
    word = []
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
 
        if part_of_speech == "名詞":
            word.append(token.surface)        
        #if part_of_speech == "動詞":        
            #word.append(token.surface)
        if part_of_speech == "形容詞":
            word.append(token.surface)        
        if part_of_speech == "形容動詞":        
            word.append(token.surface)            
    return word


tweets = pd.read_csv("tweets.csv",engine = "python")

tweets = tweets.drop("tweet_id",axis=1)
tweets = tweets.drop("timestamp",axis=1)
tweets = tweets.drop("source",axis=1)
tweets = tweets.drop("retweeted_status_id",axis=1)
tweets = tweets.drop("retweeted_status_user_id",axis=1)
tweets = tweets.drop("expanded_urls",axis=1)
tweets = tweets.drop("retweeted_status_timestamp",axis=1)
#tweets.head(20)

tweets["in_reply_to_status_id"].fillna("No_data",inplace=True)

new_tweet = []

#誰かへのリプを削除
for i in range(len(tweets)):
   if(tweets["in_reply_to_status_id"][i] == "No_data"):
      new_tweet.append(tweets["text"][i])
      
      
#RTを削除
data = []
pattern = r"RT"
for j in range(len(new_tweet)):
  if(not(re.match(pattern,new_tweet[j]))):
      data.append(new_tweet[j])

#リンクを含むツイートを削除
data_ = []
patterns = r"https"
for k in range(len(data)):
  if(not(re.search(patterns,data[k]))):
    data_.append(data[k])
      
#リプが消せ切れてなかったので
data__ = []
pattern_ = r"@"
for l in range(len(data_)):
    if(not(re.search(pattern_,data_[l]))):
        data__.append(data_[l])



tweet_data = pd.DataFrame(data__)

tweet_data.head(120)

tweets = np.array(tweet_data)
word = ""
for text in tweets:
  word += text  
text = tokenize(word)


学習させて、結果を調べる


In [50]:

model = word2vec.Word2Vec([text], size=400, min_count=20, window=15)
print(model.most_similar(positive=["高専"]))

[('ん', 0.9999973177909851), ('こと', 0.9999971389770508), ('の', 0.9999970197677612), ('人', 0.9999968409538269), ('マジ', 0.999996542930603), ('いい', 0.999996542930603), ('学校', 0.9999963045120239), ('ない', 0.9999962449073792), ('笑', 0.9999961256980896), ('...', 0.9999960064888)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
